In [ ]:
import pyaida
from pyaida import Runner, AbstractModel

# The most basic thing 
- you should be able to do is create an object and run
- the object has a system prompt and some functions including externs
- later we can load these objects from the database

In [ ]:
class MyAgent(AbstractModel):
    """You are an agent that calls the functions the user suggests to get the answer"""
    
    name: str
    description: str
        
    @classmethod
    def do_the_thing(cls, the_input:str):
        """
        This function does the thing - if the user supplies no value you can pass in any input you like as this is for testing
        
        Args:
            the_input: pass in any input to do the thing - choose any value if none supplied by user
        """
        if the_input is None:
            return "You called the function without any parameters - please call it again and supply ANY value without asking the user for help"
        return f"You have called the function with the input [{the_input}]"

from IPython.display import Markdown

Markdown(MyAgent._get_system_prompt_markdown())

## Get an instance of the runner and check its default functions
- it has runner functions and also any that are defined on the agent
- check the spec (naming must be global and openai friendly)
- and check the callable

In [ ]:
r=Runner(MyAgent)
r

In [ ]:
r.functions

In [ ]:
r.functions['90cc5_MyAgent_do_the_thing'].to_json_spec()

In [ ]:
r.functions['90cc5_MyAgent_do_the_thing'](the_input='test it')

## Next test that the agent runs and calls the function

In [ ]:
r("what parameters are specified for the function defintion of `do_the_thing`")

In [ ]:
r("please do the thing")

In [ ]:
r.messages.model_dump()

## Now we want to understand how external functions are loaded
- discover on other object
- discover on API

In [ ]:
uri = 'http://127.0.0.1:8002/openapi.json'
from pyaida.core.parsing.openapi import OpenApiSpec

s = OpenApiSpec(uri)

s._endpoint_methods

In [ ]:
"""the function manager can now use the proxy to call functions"""

from pyaida.core.lang import FunctionManager

f = FunctionManager()
"""function managers can load API proxies - by default the pyaida api is used"""
f.proxy.endpoints

In [ ]:
f.proxy.invoke_function('get_resources__get')[0]

In [ ]:
"""test that the spec matches the correct format"""
f.proxy.get_resources__get.spec

In [ ]:
"""we can access the operation as an attribute so that the proxy is more object like"""
f.proxy.get_resources__get()[0]

In [ ]:
"""we now need to check activation
1. by default the pyaida should be used
2. we should be able to qualify an API and there should be a lookup in the database to gets its url and token
"""

added_functions = f.add_functions_by_key('get_resources__get')
#cjec call , no params -> added_functions[0]()
added_functions

In [ ]:
added_functions[0].to_json_spec(), added_functions[0].hashed_qualified_name

In [ ]:
"""now check that we can also load libray functions e.g. any agent in the code base has functions"""

added_functions = f.add_functions_by_key('system.Plan.test_plan')
#cjec call , no params -> added_functions[0](context='test')
added_functions

In [ ]:
added_functions[0].to_json_spec(), added_functions[0].hashed_qualified_name

In [ ]:
"""now test it in the runner - if the function manager can add functions and they are callable
the agent should be able to also
"""
from IPython.display import Markdown

Markdown(r("Please call the function get_resources__get and tell me what you find"))

# Part 2 - reading models from the database
- we just check the basic binding and the ability to create new objects via the API
- then we swtich to a planning mode which will flex the postgres modalities 

In [14]:
from IPython.display import Markdown
import pyaida
from pyaida import Runner, AbstractModel
from pyaida.core.lang.functions import FunctionModel, Function
from pyaida.core.lang import FunctionManager
from pyaida import pg

In [15]:
#pg.repository(FunctionModel).register(plan=False)

In [28]:
pg.execute("SELECT * FROM FunctionModel")

[{'id': '8ba12db8-b753-11ef-bf11-7606330c235f',
  'description': 'some test function',
  'name': 'test_function',
  'key': 'test_function',
  'spec': {'name': 'test_function',
   'description': 'some test function',
   'parameters': {'properties': {'arg': {'description': 'description',
      'title': 'Arg',
      'type': 'string'}},
    'type': 'object'}},
  'group_id': None,
  'created_at': datetime.datetime(2024, 12, 11, 0, 4, 53, 298568),
  'updated_at': datetime.datetime(2024, 12, 11, 0, 4, 53, 298568),
  'deleted_at': None,
  'userid': None},
 {'id': '199fdfb0-b754-11ef-819b-7606330c235f',
  'description': 'some test function',
  'name': 'test_function',
  'key': 'test_function',
  'spec': {'name': 'test_function',
   'description': 'some test function',
   'parameters': {'properties': {'arg': {'description': 'description',
      'title': 'Arg',
      'type': 'string'}},
    'type': 'object'}},
  'group_id': None,
  'created_at': datetime.datetime(2024, 12, 11, 0, 8, 51, 888414),


In [25]:
import uuid

def test_function(arg:str):
    """some test function
    Args:
        arg: description
    """
    return arg
test_function.__name__
fn = FunctionModel.from_function(test_function)
fn
#pg.update_records(fns)

FunctionModel(id=UUID('fbbc4fd8-5fd2-3e15-a27c-a8e84ccb8d4f'), description='some test function', name='test_function', key='test_function', spec={'name': 'test_function', 'description': 'some test function', 'parameters': {'properties': {'arg': {'description': 'description', 'title': 'Arg', 'type': 'string'}}, 'type': 'object'}}, group_id=None)

In [26]:
pg.repository(FunctionModel).update_records(fn)

[{'id': 'fbbc4fd8-5fd2-3e15-a27c-a8e84ccb8d4f',
  'description': 'some test function',
  'name': 'test_function',
  'key': 'test_function',
  'spec': {'name': 'test_function',
   'description': 'some test function',
   'parameters': {'properties': {'arg': {'description': 'description',
      'title': 'Arg',
      'type': 'string'}},
    'type': 'object'}},
  'group_id': None,
  'created_at': datetime.datetime(2024, 12, 11, 0, 16, 14, 699477),
  'updated_at': datetime.datetime(2024, 12, 11, 0, 18, 19, 592323),
  'deleted_at': None,
  'userid': None}]

In [29]:
pg.repository(FunctionModel).load_model_from_key('test_function')

In [ ]:
op_uid = None
model = pyaida.pg.repository(FunctionModel).select(op_uid) 
print(model.Config.functions)
Markdown(model._get_system_prompt_markdown())

In [14]:
uuid.uuid1()

UUID('4c348268-b750-11ef-8f5a-7606330c235f')